# Stock Price Prediction

## The Prediction will be based on the Stock Sentiment Analysis

## Predicts if the stock will go up or down.

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/Stocks Machine Learning'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
df = pd.read_csv(r'C:\Users\atuta\Documents\Projects\Stocks Machine Learning\Combined_News_DJIA.csv', encoding = 'ISO-8859-1')
df_test = pd.read_csv(r'C:\Users\atuta\Documents\Projects\Stocks Machine Learning\3titles.csv', encoding = 'ISO-8859-1')

# encoding is used to encode some of the utf-8 characters...so use it

In [3]:
df.head()


,Date,Label,Top1,Top2,Top3
0,08/08/2008,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...
1,11/08/2008,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ..."
2,12/08/2008,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav..."
3,13/08/2008,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...
4,14/08/2008,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...


In [4]:
df_test.head()

,Date,Label,Top1,Top2,Top3
0,08/03/2021,1,"b""Tesla building 'secretive' mega-battery in T...",b'Singapore Warms to Electric Cars Two Years A...,"b'Tesla buys $1.5B in Bitcoin, will accept as ..."
1,08/03/2021,1,b'The TUI share price is rising! Should I buy ...,b'The TUI share price is very attractive.',b'Vaccination gives a hope.'
2,08/03/2021,0,b'Hennes & Mauritz : Fashion giant H&M pauses ...,b'H&M looking for stitch in time to help store...,b'H&M Faces Russian Criminal Investigation Ove...


In [5]:
df.shape

(1989, 5)

### The labels for the data are: 
0 -> stock prize will not increase on the next day.
1 -> stok prize will increse on the next day.

In [31]:
# Prepare the groups for tests. 
train_dataset = df #df[df['Date']<'20150101']
test_dataset = df_test #df[df['Date'] > '20141231']

In [32]:
train_dataset.shape

(1989, 5)

In [35]:
test_dataset.shape

(3, 5)

In [36]:
# Remove punctuation
# Replace the characters.
data = train_dataset.iloc[:, 2:27]
data.replace("[^a-zA-Z]", " ", regex=True, inplace=True)

In [37]:
#renaming column
list1 = [i for i in range(3)]
new_Index = [str(i) for i in list1]
data.columns = new_Index
data.head(5)

,0,1,2
0,b Georgia downs two Russian warplanes as cou...,b BREAKING Musharraf to be impeached,b Russia Today Columns of troops roll into So...
1,b Why wont America and Nato help us If they w...,b Bush puts foot down on Georgian conflict,b Jewish Georgian minister Thanks to Israeli ...
2,b Remember that adorable year old who sang a...,b Russia ends Georgia operation,b If we had no sexual harassment we would hav...
3,b U S refuses Israel weapons to attack Iran ...,b When the president ordered to attack Tskhinv...,b Israel clears troops who killed Reuters cam...
4,b All the experts admit that we should legalis...,b War in South Osetia pictures made by a ...,b Swedish wrestler Ara Abrahamian throws away ...


In [38]:
#converting hadlines to lower case
for index in new_Index:
    data[index] = data[index].str.lower()
data.head(2)

,0,1,2
0,b georgia downs two russian warplanes as cou...,b breaking musharraf to be impeached,b russia today columns of troops roll into so...
1,b why wont america and nato help us if they w...,b bush puts foot down on georgian conflict,b jewish georgian minister thanks to israeli ...


> Combine all the headlines to a single sentence

In [12]:
' '.join(str(x) for x in data.iloc[1, 0:25])  #example

'b why wont america and nato help us  if they wont help us now  why did we help them in iraq   b bush puts foot down on georgian conflict  b jewish georgian minister  thanks to israeli training  we re fending off russia  '

In [39]:
# Convert the sentneces into on list.

headlines = []
for row in range(0, len(data.index)):
    headlines.append(' '.join(str(x) for x in data.iloc[row, 0:25]))

In [40]:
headlines[5]

'b mom of missing gay man  too bad he s not a    year old cheerleader  then they d still be looking for him  b russia  u s  poland missile deal won t go  unpunished   b the government has been accused of creating laws that have a chilling effect on freedom of expression in the uk in a critical report from the un s committee on human rights '

Countvectorizer is used to convert all the sentences into vectors

In [41]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [42]:
# implememt BAG OF WORDS
countVector = CountVectorizer(ngram_range=(2,2))
#ngram(2,2) means it will combine the 2 words together and assign the value

train_dataset = countVector.fit_transform(headlines)

In [43]:
train_dataset[0]  # Sparse matrix ie combination of 0 and 1

<1x75159 sparse matrix of type '<class 'numpy.int64'>'
	with 29 stored elements in Compressed Sparse Row format>

### Implement Random Forest Classifier

In [44]:
RFClassifier = RandomForestClassifier(n_estimators=200, criterion='entropy')
RFClassifier.fit(train_dataset, train['Label'])


RandomForestClassifier(criterion='entropy', n_estimators=200)

In [45]:
import pickle

with open("stocks_news_model.pickle", "wb") as file:
    pickle.dump(RFClassifier, file)

### Predict for the Test Dataset

In [46]:
testTransform =[]
for row in range(0, len(test.index)):
    testTransform.append(' '.join(str(x) for x in test_dataset.iloc[row, 2:27]))
    
test_dataset = countVector.transform(testTransform)
predictions = RFClassifier.predict(test_dataset)
print(predictions)
predictions = (predictions > 0.59)
print(predictions)

[1 1 1]
[ True  True  True]


In [47]:
predictions

array([ True,  True,  True])

In [48]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [49]:
matrix = confusion_matrix(test['Label'], predictions)
print(matrix)
score = accuracy_score(test['Label'], predictions)
print(score)
report = classification_report(test['Label'], predictions)
print(report)

[[0 1]
 [0 2]]
0.6666666666666666
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           1       0.67      1.00      0.80         2

    accuracy                           0.67         3
   macro avg       0.33      0.50      0.40         3
weighted avg       0.44      0.67      0.53         3

C:\Users\atuta\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\atuta\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _war

In [51]:
from sklearn.naive_bayes import MultinomialNB

naive = MultinomialNB()
naive.fit(train_dataset, train['Label'])


MultinomialNB()

In [52]:
predictions = naive.predict(test_dataset)
predictions

array([0, 1, 0], dtype=int64)

In [53]:
matrix = confusion_matrix(test['Label'], predictions)
print(matrix)
score = accuracy_score(test['Label'], predictions)
print(score)
report = classification_report(test['Label'], predictions)
print(report)

[[1 0]
 [1 1]]
0.6666666666666666
              precision    recall  f1-score   support

           0       0.50      1.00      0.67         1
           1       1.00      0.50      0.67         2

    accuracy                           0.67         3
   macro avg       0.75      0.75      0.67         3
weighted avg       0.83      0.67      0.67         3

